In [1]:
include <- function(pkg) {
  if (!suppressMessages(require(pkg, character.only = TRUE)))
    install.packages(pkg, character.only = TRUE)
  suppressMessages(library(pkg, pkg, character.only = TRUE))
}
include("ShortRead")
include("plyr")
include("magick")

In [2]:
BASE_PATH_NR1 = "/data/samples/AIRR-Seq/OURS/S3987Nr1"
BASE_PATH_NR2 = "/data/samples/AIRR-Seq/OURS/S3987Nr2"

SAMPLES = list(
    list(base_path=BASE_PATH_NR1, name="S3987Nr1-PBMC1_heavy"),
    list(base_path=BASE_PATH_NR1, name="S3987Nr1-PBMC1_light"),
#     list(base_path=BASE_PATH_NR1, name="S3987Nr1-RAMOS_heavy"),
    list(base_path=BASE_PATH_NR1, name="S3987Nr1-RAMOS_light")
#     list(base_path=BASE_PATH_NR2, name="S3987Nr2-PBMC1_heavy"),
#     list(base_path=BASE_PATH_NR2, name="S3987Nr2-PBMC1_light"),
#     list(base_path=BASE_PATH_NR2, name="S3987Nr2-RAMOS_heavy"),
#     list(base_path=BASE_PATH_NR2, name="S3987Nr2-RAMOS_light")
)

EXCLUDE_FROM_DIVERSITY_ANALYSIS = c(
    "S3987Nr1-RAMOS_heavy"
)

In [3]:
# Helpers
get_path_sample <- function(sample_name, base_path = BASE_PATH) {
    return(paste0(base_path, "/", sample_name))
}

get_path_fastq <- function(sample_name, type = "raw", base_path = BASE_PATH) {
    
    if (type == "raw") {
        return(
            paste0(
                get_path_sample(sample_name, base_path),
                "/", sample_name, ".1.fastq"
            )
        )
    } else if (type == "primer_pass") {
        suffix = ".1_primers-pass.fastq"
    } else if (type == "pair_pass"){
        suffix = ".1_primers-pass_pair-pass.fastq"
    } else if (type == "under2") {
        suffix = "_under-2.fastq"
    } else if (type == "atleast2") {
        suffix = "_atleast-2.fastq"
    } else {
        stop(paste0("Unsupported fastq file type: ", type))
    }
    
    return(
        paste0(
            get_path_sample(sample_name, base_path),
            "/", "presto",
            "/", sample_name, suffix
        )
    )
}

get_path_igblast <- function(sample_name, base_path = BASE_PATH) {
    return(
        paste0(
            get_path_sample(sample_name, base_path), 
            "/", "changeo_igblast", 
            "/", sample_name, "_db-pass_with_translation.tsv"
        )
    )
}

get_path_clones <- function(sample_name, clone_file = "groups", base_path = BASE_PATH) {
    
    if (clone_file == "groups") {
        suffix = "_vjl_groups.tsv"
    } else if (clone_file == "clones") {
        suffix = "_with_clones.tsv"
    } else if (clone_file == "counts") {
        return(
            paste0(
                get_path_sample(sample_name, base_path),
                "/", "clones", 
                "/", "diversity",
                "/", sample_name, "_clone_counts.tsv"
            )
        )
    } else {
        stop(paste0("Unknown clone_file type: ", clone_file))
    }
    
    return(
        paste0(
            get_path_sample(sample_name, base_path),
            "/", "clones", 
            "/", sample_name, suffix
        )
    )
}

get_path_coverages <- function(sample_name, base_path = BASE_PATH) {
    return(
        paste0(
            get_path_sample(sample_name, base_path),
            "/", "clones",
            "/", "diversity",
            "/", sample_name, "_coverages.tsv"
        )
    )
}

get_path_diversity <- function(sample_name, base_path = BASE_PATH) {
    return(
        paste0(
            get_path_sample(sample_name, base_path),
            "/", "clones",
            "/", "diversity",
            "/", sample_name, "_diversity.tsv"
        )
    )
}

get_path_pngs <- function(sample_name, img_type = "abundancy", base_path = BASE_PATH) {
    
    if (img_type == "abundancy") {
        suffix = "_abundancy_curve.png"
    } else if (img_type == "diversity") {
        suffix = "_diversity.png"
    } else if (img_type == "clones") {
        return(
            paste0(
                get_path_sample(sample_name, base_path),
                "/", "clones",
                "/", sample_name, "_with_clones.png"
            )
        )
    } else {
        stop("Image type = ", img_type, " is not supported.")
    }
    
    return(
        paste0(
            get_path_sample(sample_name, base_path),
            "/", "clones",
            "/", "diversity",
            "/", sample_name, suffix
        )
    )
}

read_tsv <- function(filepath) {
    return(read.csv(filepath, sep='\t'))
}

### Characterize sequencing results for each sample

In [5]:
# fp = get_path_fastq(SAMPLES[[1]]$name, type = "atleast2", SAMPLES[[1]]$base_path)
# print(fp)
# qa_summary = ShortRead::qa(fp, type = "fastq")

[1] "/data/samples/AIRR-Seq/OURS/S3987Nr1/S3987Nr1-PBMC1_heavy/presto/S3987Nr1-PBMC1_heavy_atleast-2.fastq"


In [6]:
# report_html(qa_summary, "test.html")

class: FastqQA(10)
QA elements (access with qa[["elt"]]):
  readCounts: data.frame(1 3)
  baseCalls: data.frame(1 5)
  readQualityScore: data.frame(512 4)
  baseQuality: data.frame(95 3)
  alignQuality: data.frame(1 3)
  frequentSequences: data.frame(50 4)
  sequenceDistribution: data.frame(22 4)
  perCycle: list(2)
    baseCall: data.frame(1629 4)
    quality: data.frame(5857 5)
  perTile: list(2)
    readCounts: data.frame(0 4)
    medianReadQualityScore: data.frame(0 4)
  adapterContamination: data.frame(1 1)

In [12]:
count_reads_fastq <- function(fastq_path) {
    return(
        ShortRead::countFastq(fastq_path)$records
    )
}

In [8]:
res <- sapply(
    SAMPLES,
    function(sample) {
        counts = cbind(
            sample$name,
            count_reads_fastq(get_path_fastq(sample$name, type = "raw", sample$base_path)),
            count_reads_fastq(get_path_fastq(sample$name, type = "primer_pass", sample$base_path)),
            count_reads_fastq(get_path_fastq(sample$name, type = "pair_pass", sample$base_path)),
            count_reads_fastq(get_path_fastq(sample$name, type = "under2", sample$base_path)),
            count_reads_fastq(get_path_fastq(sample$name, type = "atleast2", sample$base_path))
        )
        return(counts)
    }
)

res <- data.frame(t(res))
colnames(res) = c("sample_name", "raw", "primer_pass", "pair_pass", "under2", "atleast2")
res

,sample_name,raw,primer_pass,pair_pass,under2,atleast2
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,S3987Nr1-PBMC1_heavy,293087,293087,293087,127350,14008
2,S3987Nr1-PBMC1_light,264459,264459,264459,108203,13384
3,S3987Nr1-RAMOS_heavy,160952,160952,160952,28,5
4,S3987Nr1-RAMOS_light,69991,69991,69991,25974,3791
5,S3987Nr2-PBMC1_heavy,4130140,4130140,4130140,1336389,187452
6,S3987Nr2-PBMC1_light,3756935,3756935,3756935,1099571,170346


In [9]:
res

sample_name,raw,primer_pass,pair_pass,under2,atleast2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
S3987Nr1-PBMC1_heavy,293087,293087,293087,127350,14008
S3987Nr1-PBMC1_light,264459,264459,264459,108203,13384
S3987Nr1-RAMOS_heavy,160952,160952,160952,28,5
S3987Nr1-RAMOS_light,69991,69991,69991,25974,3791
S3987Nr2-PBMC1_heavy,4130140,4130140,4130140,1336389,187452
S3987Nr2-PBMC1_light,3756935,3756935,3756935,1099571,170346
S3987Nr2-RAMOS_heavy,2273707,2273707,2273707,511006,82177


### Compare clone counts

In [68]:
columnar_dataframe_builder <- function(values_vec, colnames) {
    df = data.table::transpose(data.frame(values_vec))
    colnames(df) = colnames
    return(df)
}

aggregate_tsvs <- function(samples, path_builder, mode = "headers", nrows = 5, processor = NULL) {
    
    if (!(mode %in% c("headers", "processor"))) {
        stop(paste0("Mode not supported: ", mode))
    }
    
    df_res = data.frame()
    for (i in 1:length(samples)) {
        
        sample = samples[[i]]
        filepath = path_builder(sample$name, sample$base_path)
        
        if (file.exists(filepath)) {
            
            if (mode == "headers") {
                tsv_head = head(read_tsv(filepath), n = nrows)
                df_annotated = cbind(sample$name, tsv_head)
                df_res = plyr::rbind.fill(df_res, df_annotated)
            } else if (mode == "processor") {
                tsv = read_tsv(filepath)
                df_processed = processor(tsv)
                df_annotated = cbind(sample$name, df_processed)
                df_res = plyr::rbind.fill(df_res, df_annotated)
            }
        
        } else {
            print(paste0("Filepath not found: ", filepath))
        }
    }
    return(df_res)
}

In [57]:
clone_counts = aggregate_tsvs(
    SAMPLES,
    function(name, base_path) {get_path_clones(name, "counts", base_path)},
    nrows = 10
)
clone_counts

sample$name,clone_id,seq_count,copy_count,seq_freq,copy_freq
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
S3987Nr1-PBMC1_heavy,556,1628,24866,0.1248370524,1.944251e-01
S3987Nr1-PBMC1_heavy,374,718,10748,0.0550571275,8.403769e-02
S3987Nr1-PBMC1_heavy,900,781,8913,0.0598880454,6.968998e-02
S3987Nr1-PBMC1_heavy,453,533,8079,0.0408710988,6.316901e-02
S3987Nr1-PBMC1_heavy,1316,536,7377,0.0411011426,5.768013e-02
S3987Nr1-PBMC1_heavy,1216,333,3103,0.0255348516,2.426209e-02
S3987Nr1-PBMC1_heavy,1259,220,1913,0.0168698719,1.495758e-02
S3987Nr1-PBMC1_heavy,942,206,1856,0.0157963346,1.451190e-02
S3987Nr1-PBMC1_heavy,1144,175,1595,0.0134192163,1.247117e-02


### Abundancy and diversity comparisons among samples

In [58]:
coverages_df = aggregate_tsvs(
    SAMPLES, 
    function(sample_name, base_path) {get_path_coverages(sample_name, base_path)}, 
    nrows = 10
)
coverages_df

sample$name,X1.order,coverages
<chr>,<int>,<dbl>
S3987Nr1-PBMC1_heavy,1,0.9670290
S3987Nr1-PBMC1_heavy,2,0.9999302
S3987Nr1-PBMC1_heavy,3,0.9999999
S3987Nr1-PBMC1_heavy,4,1.0000000
S3987Nr1-PBMC1_heavy,5,1.0000000
S3987Nr1-PBMC1_heavy,6,1.0000000
S3987Nr1-PBMC1_heavy,7,1.0000000
S3987Nr1-PBMC1_heavy,8,1.0000000
S3987Nr1-PBMC1_heavy,9,1.0000000


In [73]:
diversity <- read_tsv(
    get_path_diversity(SAMPLES[[1]]$name, SAMPLES[[1]]$base_path)
)

interpret_diversity <- function(diversity_df) {

    extract_div_by_q <- function(q) {diversity_df[diversity$q == q, ]$d}
    
    species_richness <- extract_div_by_q(0)
    D_1 <- extract_div_by_q(1)
    D_2 <- extract_div_by_q(2)
    D_3 <- extract_div_by_q(3)
    D_4 <- extract_div_by_q(4)
    shannon_index <- log(D_1, 2)
    gini_simpson_index <- 1 - 1/D_2
    
    res = columnar_dataframe_builder(
        values_vec = c(species_richness, D_1, D_2, D_3, D_4, shannon_index, gini_simpson_index),
        colnames = c("Species_richness_D_0", "D_1", "D_2", "D_3", "D_4", "Shannon_uncertainty", "Gini_simpson_prob")
    )
    
#     res = data.frame(
#         c("Species_richness_D_0", "D_1", "D_2", "D_3", "D_4", "Shannon_uncertainty", "Gini_simpson_prob"), 
#         c(species_richness, D_1, D_2, D_3, D_4, shannon_index, gini_simpson_index)
#     )
    return(res)
}

In [79]:
# head(diversity)
# interpret_diversity(diversity)
aggregate_tsvs(
    SAMPLES,
    get_path_diversity,
    mode = "processor",
    processor = interpret_diversity
)

sample$name,Species_richness_D_0,D_1,D_2,D_3,D_4,Shannon_uncertainty,Gini_simpson_prob
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
S3987Nr1-PBMC1_heavy,1254.24,213.581173,35.692533,20.000165,15.449315,7.7386407,0.97198294
S3987Nr1-PBMC1_light,540.08,58.269088,20.190402,13.385689,10.718324,5.8646588,0.95047152
S3987Nr1-RAMOS_light,27.84,1.109768,1.024784,1.018536,1.016459,0.1502587,0.02418428


### Join plots

In [17]:
# magick::image_append(magick::image_scale(img, "100"), stack = TRUE)

In [21]:
img1 = magick::image_read(get_path_pngs(SAMPLES[[1]]$name, "abundancy", SAMPLES[[1]]$base_path))
img2 = magick::image_read(get_path_pngs(SAMPLES[[2]]$name, "abundancy", SAMPLES[[2]]$base_path))
img3 = magick::image_read(get_path_pngs(SAMPLES[[3]]$name, "abundancy", SAMPLES[[3]]$base_path))
comb_img = magick::image_append(c(img1, img2, img3), stack=TRUE)
magick::image_write(comb_img, path = "tiger.png", format = "png")

In [23]:
img1 = magick::image_read(get_path_pngs(SAMPLES[[1]]$name, "diversity", SAMPLES[[1]]$base_path))
img2 = magick::image_read(get_path_pngs(SAMPLES[[2]]$name, "diversity", SAMPLES[[2]]$base_path))
img3 = magick::image_read(get_path_pngs(SAMPLES[[3]]$name, "diversity", SAMPLES[[3]]$base_path))
# img4 = magick::image_read(get_path_pngs(SAMPLES[[4]]$name, "clones", SAMPLES[[4]]$base_path))

comb_img = comb_img = magick::image_append(c(img1, img2, img3), stack=TRUE)
magick::image_write(comb_img, path = "tiger.png", format = "png")

In [22]:
img1 = magick::image_read(get_path_pngs(SAMPLES[[1]]$name, "clones", SAMPLES[[1]]$base_path))
img2 = magick::image_read(get_path_pngs(SAMPLES[[2]]$name, "clones", SAMPLES[[2]]$base_path))
img3 = magick::image_read(get_path_pngs(SAMPLES[[3]]$name, "clones", SAMPLES[[3]]$base_path))
# img4 = magick::image_read(get_path_pngs(SAMPLES[[4]]$name, "clones", SAMPLES[[4]]$base_path))

comb_img = comb_img = magick::image_append(c(img1, img2, img3), stack=TRUE)
magick::image_write(comb_img, path = "tiger.png", format = "png")